In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
!nvidia-smi

Sat Mar  2 03:05:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install transformers

In [4]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [6]:
df=pd.read_csv("/content/modified_food - clean_anno_reduced.csv")

In [7]:
df.shape

(572, 14)

In [8]:
train1_df = df[:520]
test_df = df[520:]

In [9]:
train1_df.head()

,Item,Diet,Protein,Carbs,Fats,Sweet,Bitter,Spicy,Sour,Dessert,Main-course,Starter,North-indian,South-indian
0,apple_pie,0,0,1,1,1,0,0,0,1,0,0,0,0
1,baby_back_ribs,1,1,0,1,0,0,1,0,0,1,0,0,0
2,beef_tacos,1,1,1,1,0,0,1,0,0,1,0,0,0
3,beet_salad,0,1,0,0,0,1,0,1,0,1,0,0,0
4,bread_pudding,1,1,1,1,1,0,0,0,1,0,0,0,0


In [10]:
train1_df.columns

Index(['Item', 'Diet', 'Protein', 'Carbs', 'Fats', 'Sweet', 'Bitter', 'Spicy',
       'Sour', 'Dessert', 'Main-course', 'Starter', 'North-indian',
       'South-indian'],
      dtype='object')

In [11]:
train1_df.head()

,Item,Diet,Protein,Carbs,Fats,Sweet,Bitter,Spicy,Sour,Dessert,Main-course,Starter,North-indian,South-indian
0,apple_pie,0,0,1,1,1,0,0,0,1,0,0,0,0
1,baby_back_ribs,1,1,0,1,0,0,1,0,0,1,0,0,0
2,beef_tacos,1,1,1,1,0,0,1,0,0,1,0,0,0
3,beet_salad,0,1,0,0,0,1,0,1,0,1,0,0,0
4,bread_pudding,1,1,1,1,1,0,0,0,1,0,0,0,0


In [12]:
# train1_df['Item'] = train1_df['Item'] + ' ' + train1_df['Ingredients']

# Drop the Ingredients column
# train1_df.drop(columns=['Ingredients'], inplace=True)



In [13]:
train1_df = train1_df[:350]

In [14]:
train1_df.head()

,Item,Diet,Protein,Carbs,Fats,Sweet,Bitter,Spicy,Sour,Dessert,Main-course,Starter,North-indian,South-indian
0,apple_pie,0,0,1,1,1,0,0,0,1,0,0,0,0
1,baby_back_ribs,1,1,0,1,0,0,1,0,0,1,0,0,0
2,beef_tacos,1,1,1,1,0,0,1,0,0,1,0,0,0
3,beet_salad,0,1,0,0,0,1,0,1,0,1,0,0,0
4,bread_pudding,1,1,1,1,1,0,0,0,1,0,0,0,0


In [15]:
target_list = list(train1_df.columns)[1:]
target_list

['Diet',
 'Protein',
 'Carbs',
 'Fats',
 'Sweet',
 'Bitter',
 'Spicy',
 'Sour',
 'Dessert',
 'Main-course',
 'Starter',
 'North-indian',
 'South-indian']

In [16]:
# hyperparameters
MAX_LEN = 40
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 50
LEARNING_RATE = 1e-05

In [17]:
from transformers import BertTokenizer, BertModel

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [19]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['Item']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [20]:
train_size = 0.8
train_df = train1_df.sample(frac=train_size, random_state=2).reset_index(drop=True)
val_df = train1_df.drop(train_df.index).reset_index(drop=True)
val_df.index

RangeIndex(start=0, stop=70, step=1)

In [21]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
print(len(valid_dataset),len(train_dataset))

70 280


In [22]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
len(val_data_loader)

7

In [32]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device

device(type='cuda')

In [33]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [34]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 13)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [35]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [36]:
val_targets=[]
val_outputs=[]

In [37]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      #save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        #save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [38]:
ckpt_path = "/content/drive/MyDrive/ccokr/"
best_model_path = "/content/drive/MyDrive/ccokr/best_model.pt"

In [44]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.020181 	Average Validation Loss: 0.075894
Validation loss decreased (inf --> 0.075894).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.017931 	Average Validation Loss: 0.068445
Validation loss decreased (0.075894 --> 0.068445).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [40]:
target_list

['Diet',
 'Protein',
 'Carbs',
 'Fats',
 'Sweet',
 'Bitter',
 'Spicy',
 'Sour',
 'Dessert',
 'Main-course',
 'Starter',
 'North-indian',
 'South-indian']

In [41]:
import pickle
# Save the trained model using pickle
with open('model-cpu.pkl', 'wb') as file:
    pickle.dump(trained_model, file)

# Load the saved model from the file
with open('model-cpu.pkl', 'rb') as file:
    loaded_model = pickle.load(file)


In [45]:
# testing
example = "idli"
# test_df['Item'][566]
print("Item : ",example)
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)
loaded_model.eval()
output_list=[example]
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    #print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
    #print(final_output)
    if(final_output[0][8]>final_output[0][9] and  final_output[0][8]>final_output[0][10]):
        final_output[0][8]=1
        final_output[0][9],final_output[0][10]=0,0
    elif(final_output[0][9]>final_output[0][8] and  final_output[0][9]>final_output[0][10]):
      final_output[0][9]=1
      final_output[0][8],final_output[0][10]=0,0
    else:
      final_output[0][10]=1
      final_output[0][9],final_output[0][8]=0,0
    # if(final_output[0][1]>0.8):
    #   output_list.append("Non-veg")
    # else:
    #   output_list.append("Veg")
    for i in range(1,len(final_output[0])):
      if (final_output[0][i] >=0.5):
        print(target_list[i])
        output_list.append(target_list[i])
print(output_list)


Item :  idli
Protein
Carbs
Fats
Spicy
Main-course
South-indian
['idli', 'Protein', 'Carbs', 'Fats', 'Spicy', 'Main-course', 'South-indian']


In [43]:
# Extracting food name and info
foodname = output_list[0]
info = ' '.join(output_list[1:])  # Joining all elements except the first one with a space

# Sample price and other information
price = "₹25.00"
s,m,d=0,0,0
if("Starter" in output_list):
  s=1
elif("Main-course" in output_list):
  m=1
else:
  d=1
# Mapping food categories to sections (assuming all are main-course)
result = {
    'output': {
        'foodname': foodname,
        'price': price,
        'info': info,
        's': str(s),
        'm': str(m),
        'd': str(d)
    }
}

print(result)


{'output': {'foodname': 'gulab jamun', 'price': '₹25.00', 'info': 'Protein Carbs Fats Spicy Main-course South-indian', 's': '0', 'm': '1', 'd': '0'}}
